In [11]:
!pip install --quiet -U  pyserini jsonlines

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
DATA_PATH = './data'
!mkdir data && cp /content/drive/MyDrive/DATASETS/MachineLearning/topicmodeling/lenta-ru-filtered.csv data

## Библиотеки

In [52]:
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from matplotlib import gridspec

from tqdm.notebook import tqdm
import numpy as np
import  pandas as pd
import seaborn as sns
import torch
import scipy
import json
import re
import os
import jsonlines

from pyserini.search import SimpleSearcher

## Загружаем данные

In [7]:
data = pd.read_csv('data/lenta-ru-filtered.csv')

In [8]:
data

,text,tags,len,date
0,С 1 сентября на всей территории России вводитс...,Все,1654,31-08-1999
1,"По сведениям миссии ООН, передаваемым РИА ""Нов...",Все,1086,31-08-1999
2,15 представителей национал-большевистской парт...,Все,1219,31-08-1999
3,Намеченная на сегодняшний день церемония вступ...,Все,3094,31-08-1999
4,"На юге Киргизии, а именно в Баткенском и Чон-А...",Все,1354,31-08-1999
...,...,...,...,...
863280,Популярное место среди туристов в Мурманской о...,Россия,1231,11-09-2020
863281,Рейтинги от международного рейтингового агентс...,NaN,1425,11-09-2020
863282,Российские ученые нашли в Якутии новый подвид ...,События,1299,11-09-2020
863283,Для указания коронавируса как причины смерти ч...,Общество,2061,11-09-2020


## Описания поисковика

## Генерация документов

In [36]:
COLLECTION_DIR = 'data/collection/lenta'
if not os.path.exists(COLLECTION_DIR):
    os.makedirs(COLLECTION_DIR)

collection = data.values

bath_size = 32768
for ind in tqdm(range(0, len(collection), bath_size)):
    files = [{'id': ind+i, 
              'contents': doc[0],
              'tag': doc[1],
              'date': doc[-1]}
             for i, doc in enumerate(collection[ind:ind+bath_size])]
    with jsonlines.open(os.path.join(COLLECTION_DIR, 
                                     '{}.jsonl'.format(ind)), 
                        mode='w') as writer:
        writer.write_all(files)

  0%|          | 0/27 [00:00<?, ?it/s]

In [39]:
with open(os.path.join(COLLECTION_DIR, 
                       '{}.jsonl'.format(ind))) as f:
  print('\n'.join(f.read().splitlines()[-5:]))

{"id": 863280, "contents": "Популярное место среди туристов в Мурманской области завалили мусором и навозом. Об этом сообщает местное издание «Мурманский вестник». По данным портала, речь идет о Лавнинских водопадах в Кольском районе региона, ставших одним из излюбленных мест отдыха северян, где можно «искупаться и сделать красивые фотографии». Местные жители жалуются, что туристы не убирают за собой мусор и пластиковые бутылки, а недавно там появились огромные лужи навоза. «На всем пути к Лавне \"аромат\" такой, аж оторопь берет... Особое негодование вызывают болотного вида лужи навоза, смердящего и отравляющего воду. А ведь лет пять назад мы набирали чайник прямо из водопада, не боялись отравиться нечистотами», — рассказала жительница района Лиза Титова. Откуда взялся навоз возле водопадов, неизвестно, но люди предполагают, что во всем могут быть виноваты фермеры, сливающие отходы на территорию курорта. В июне сообщалось, что жители английского города Бакстон подговорили фермеров заб

## Построение индекса

In [40]:
INDEX_DIR = 'data/index/lenta'
if not os.path.exists(INDEX_DIR):
    os.makedirs(INDEX_DIR)

In [43]:
!python -m pyserini.index -collection JsonCollection -generator DefaultLuceneDocumentGenerator \
 -threads 16 -input {COLLECTION_DIR} \
 -index {INDEX_DIR} -storePositions -storeDocvectors -storeRaw

2021-03-18 21:09:16,821 INFO  [main] index.IndexCollection (IndexCollection.java:655) - Setting log level to INFO
2021-03-18 21:09:16,825 INFO  [main] index.IndexCollection (IndexCollection.java:658) - Starting indexer...
2021-03-18 21:09:16,825 INFO  [main] index.IndexCollection (IndexCollection.java:659) - ============ Loading Parameters ============
2021-03-18 21:09:16,826 INFO  [main] index.IndexCollection (IndexCollection.java:660) - DocumentCollection path: data/collection/lenta
2021-03-18 21:09:16,827 INFO  [main] index.IndexCollection (IndexCollection.java:661) - CollectionClass: JsonCollection
2021-03-18 21:09:16,828 INFO  [main] index.IndexCollection (IndexCollection.java:662) - Generator: DefaultLuceneDocumentGenerator
2021-03-18 21:09:16,828 INFO  [main] index.IndexCollection (IndexCollection.java:663) - Threads: 16
2021-03-18 21:09:16,829 INFO  [main] index.IndexCollection (IndexCollection.java:664) - Stemmer: porter
2021-03-18 21:09:16,829 INFO  [main] index.IndexCollecti

## Используем поисковый индекс

### BM-25

Пусть дан запрос $Q$, содержащий слова $q_1, ..., q_n$, тогда функция BM25 даёт следующую оценку релевантности документа $D$ запросу $Q$:
$$\text{score}(D,Q) = \sum_{i=1}^{n} \text{IDF}(q_i) \cdot \frac{f(q_i, D) \cdot (k_1 + 1)}{f(q_i, D) + k_1 \cdot (1 - b + b \cdot \frac{|D|}{\text{avgdl}})},$$
где $f(q_i, D)$ есть частота слова $q_i$ в документе $D$, $|D|$ есть длина документа (количество слов в нём), а $vgdl$ — средняя длина документа в коллекции. $k_1$ и $b$ — свободные коэффициенты, обычно их выбирают как $k_1 = 2.0$ и $b = 0.75$.

In [62]:
searcher = SimpleSearcher(INDEX_DIR)
searcher.set_bm25()
hits = searcher.search('Трамп', k=1)

for i in range(len(hits)):
    doc = searcher.doc(hits[i].docid)
    print(doc.raw())

{
  "id" : 110803,
  "contents" : "58-летний американский миллиардер Дональд Трамп и 33-летняя словенская супермодель Мелания Кнаусс должны пожениться 22 января в Палм-Бич, шт. Флорида. На церемонию приглашены известные политические деятели и звезды Голливуда. Свадьбу намечено гулять с большим размахом, но платить за нее Трамп не собирается. Приглашения на свадебную церемонию были разосланы известным политическим деятелям, а также звездам Голливуда. Приглашения уже приняли экс-чемпион мира по боксу Мохаммед Али, певец Элтон Джон, актриса Кэтрин Зета-Джонс и актер Майкл Дуглас. Две бывшие жены Трампа, которым он также направил приглашения, прийти на свадьбу отказались. Несмотря на то, что Трамп считается очень богатым человеком, платить за свою свадьбу он не собирается. Как сказал Трамп: \"Когда я сказал, что женюсь, мне все стали наперебой предлагать свои услуги бесплатно. С точки зрения бизнеса - это очень правильный ход\". Единственное, что может огорчить гостей свадебной вечеринки -

### QLD

In [63]:
searcher = SimpleSearcher(INDEX_DIR)
searcher.set_qld(,)
hits = searcher.search('Трамп', k=1)

for i in range(len(hits)):
    doc = searcher.doc(hits[i].docid)
    print(doc.raw())

{
  "id" : 110803,
  "contents" : "58-летний американский миллиардер Дональд Трамп и 33-летняя словенская супермодель Мелания Кнаусс должны пожениться 22 января в Палм-Бич, шт. Флорида. На церемонию приглашены известные политические деятели и звезды Голливуда. Свадьбу намечено гулять с большим размахом, но платить за нее Трамп не собирается. Приглашения на свадебную церемонию были разосланы известным политическим деятелям, а также звездам Голливуда. Приглашения уже приняли экс-чемпион мира по боксу Мохаммед Али, певец Элтон Джон, актриса Кэтрин Зета-Джонс и актер Майкл Дуглас. Две бывшие жены Трампа, которым он также направил приглашения, прийти на свадьбу отказались. Несмотря на то, что Трамп считается очень богатым человеком, платить за свою свадьбу он не собирается. Как сказал Трамп: \"Когда я сказал, что женюсь, мне все стали наперебой предлагать свои услуги бесплатно. С точки зрения бизнеса - это очень правильный ход\". Единственное, что может огорчить гостей свадебной вечеринки -

### RM3

In [64]:
searcher = SimpleSearcher(INDEX_DIR)
searcher.set_rm3(,)
hits = searcher.search('Трамп', k=1)

for i in range(len(hits)):
    doc = searcher.doc(hits[i].docid)
    print(doc.raw())

{
  "id" : 720918,
  "contents" : "Президент США Дональд Трамп в интервью телеканалу CNBC признался, что готов обложить протекционистскими пошлинами практически весь импорт из Китая в США. В настоящий момент власти США уже ввели пошлины против китайского импорта на сумму в 34 миллиарда долларов, в ближайшее время эту сумму доведут до 50 миллиардов. «Я готов дойти до 500 [миллиардов]», — заявил Трамп ведущему телеканала Джо Кернену. «Я делаю это не из-за политики, я делаю это, потому что это правильная вещь для нашей страны. Китай давит на нас уже очень долгое время», — отметил президент США. По его словам, Вашингтон обладает рядом преимуществ над Пекином, которые готов использовать, в частности, в торговой и монетарной политике. «Я не хочу, чтобы они боялись, я хочу, чтобы они хорошо себя вели. Мне очень нравится лидер КНР, но это просто нечестно», — утверждает президент. Трамп признал, что ему не нравится, что делает Китай. Как отмечает телеканал со ссылкой на государственную статисти